In [13]:
%load_ext autoreload
%autoreload 2

import sys  
import pickle 
sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn

run_settings = '/data/zach_work/side_projects/simple_geodyn_run.yaml'

### Load the data into an object
RunObject = Pygeodyn(run_settings)
RunObject.RUN_make_OrbitCloud_trajectories()




# Reformat the HASDM data to be fixed width

In [14]:
# import numpy as np
# import pandas as pd


# hasdm_input = pd.read_csv('/data/zach_work/side_projects/OrbitCloud_Step10_2018313_hasdm.csv', 
#                             dtype=object,
#                             names = ['YYMMDDHHMMSS',
#                                      'lon',
#                                      'lat',
#                                      'alt',
#                                      'rho',
#                                          ],
#                             sep = ',',#'\s+',
#                             )


# # HASDM_orbitcloud_file = '/data/zach_work/side_projects/HASDM_OrbitCloud2018313.csv'

# # f = open(HASDM_orbitcloud_file, "w")
# # f.write("\n")
# # f.close()


# # with open(HASDM_orbitcloud_file, 'r+') as file:

# #     for ii, valval in enumerate(hasdm_input['YYMMDDHHMMSS']):
# #         file.write(f"{int(hasdm_input['YYMMDDHHMMSS'][ii])}   {float(hasdm_input['lon'][ii]):8.4f}   {float(hasdm_input['lat'][ii]):8.4f}   {float(hasdm_input['alt'][ii]):8.4f}   {float(hasdm_input['rho'][ii]):15.8e} \n")



In [15]:
# hasdm_input

In [16]:
%load_ext autoreload
%autoreload 2

import sys  
import pickle 
sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Run GEODYN with HASDM:


In [5]:
# %load_ext autoreload
# %autoreload 2

# import sys  
# import pickle 
# sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
# from PYGEODYN import Pygeodyn

# run_settings = '/data/zach_work/side_projects/Run_HASDM_settings.yaml'

# ### Load the data into an object
# RunObject = Pygeodyn(run_settings)
# RunObject.RUN_GEODYN()




In [6]:

models_list = [
               'msis2'    ,
               'dtm87'    ,
               'jaachia71',
               'tiegcm_oc',
               ] 

Obj_Geodyn = {}
for imodel,val in enumerate( models_list ):  

    dir_save = '/data/data_geodyn/results/icesat2/'+val+'/'+val+'_acceloff_valid_ctrlrun'+'/'
    filehandler = open(dir_save+'DenModelCompares313_327'+val+'.pkl', 'rb') 
    Obj_Geodyn[val] = pickle.load(filehandler)
    
    filehandler.close()

In [7]:
%load_ext autoreload
%autoreload 2

import sys  
import pickle 
sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn


for i,val in enumerate( ['hasdm_oc'] ): 
    run_settings = '/data/zach_work/side_projects/Run_HASDM_settings.yaml'

    ### Load the data into an object
    Obj_Geodyn[val] = Pygeodyn(run_settings)
    Obj_Geodyn[val].getData()
    Obj_Geodyn[val].post_getData_ValidationMetrics()

    
    
#     #### Pickle the object to save it
#     print('Saving pickle')
#     dir_save = '/data/data_geodyn/results/icesat2/'+val+'/'+val+'_acceloff_valid_ctrlrun'+'/'
#     filehandler = open(dir_save+'DenModelCompares313_327'+val+'.pkl', 'wb') 
#     pickle.dump(Obj_Geodyn, filehandler)
#     filehandler.close()
#     Obj_Geodyn = 0
#     print('Saved pickle')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


KeyError: 'cd_model'

In [9]:
for imodel,val in enumerate( models_list ):  
    Obj_Geodyn[val].__dict__['global_params']['arc_input'] = Obj_Geodyn['hasdm_oc'].__dict__['global_params']['arc_input'][:2]

KeyError: 'hasdm_oc'

In [10]:
Obj_Geodyn['tiegcm_oc'].__dict__['global_params']['arc_input'][:2]

['2018.313', '2018.314']

# Plots

In [11]:
import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd

import numpy as np

config = dict({
                'displayModeBar': True,
                'responsive': False,
                'staticPlot': False,
                'displaylogo': False,
                'showTips': False,
                })



import numpy as np
import pandas as pd
import os

# px.colors.colorscale_to_colors()
# plotly.colors.PLOTLY_SCALES["Viridis"]

def get_color(colorscale_name, loc):
    from _plotly_utils.basevalidators import ColorscaleValidator
    # first parameter: Name of the property being validated
    # second parameter: a string, doesn't really matter in our use case
    cv = ColorscaleValidator("colorscale", "")
    # colorscale will be a list of lists: [[loc1, "rgb1"], [loc2, "rgb2"], ...] 
    colorscale = cv.validate_coerce(colorscale_name)
    
    if hasattr(loc, "__iter__"):
        return [get_continuous_color(colorscale, x) for x in loc]
    return get_continuous_color(colorscale, loc)
        

# Identical to Adam's answer
import plotly.colors
from PIL import ImageColor

def get_continuous_color(colorscale, intermed):
    """
    Plotly continuous colorscales assign colors to the range [0, 1]. This function computes the intermediate
    color for any value in that range.

    Plotly doesn't make the colorscales directly accessible in a common format.
    Some are ready to use:
    
        colorscale = plotly.colors.PLOTLY_SCALES["Greens"]

    Others are just swatches that need to be constructed into a colorscale:

        viridis_colors, scale = plotly.colors.convert_colors_to_same_type(plotly.colors.sequential.Viridis)
        colorscale = plotly.colors.make_colorscale(viridis_colors, scale=scale)

    :param colorscale: A plotly continuous colorscale defined with RGB string colors.
    :param intermed: value in the range [0, 1]
    :return: color in rgb string format
    :rtype: str
    """
    if len(colorscale) < 1:
        raise ValueError("colorscale must have at least one color")

    hex_to_rgb = lambda c: "rgb" + str(ImageColor.getcolor(c, "RGB"))

    if intermed <= 0 or len(colorscale) == 1:
        c = colorscale[0][1]
        return c if c[0] != "#" else hex_to_rgb(c)
    if intermed >= 1:
        c = colorscale[-1][1]
        return c if c[0] != "#" else hex_to_rgb(c)

    for cutoff, color in colorscale:
        if intermed > cutoff:
            low_cutoff, low_color = cutoff, color
        else:
            high_cutoff, high_color = cutoff, color
            break

    if (low_color[0] == "#") or (high_color[0] == "#"):
        # some color scale names (such as cividis) returns:
        # [[loc1, "hex1"], [loc2, "hex2"], ...]
        low_color = hex_to_rgb(low_color)
        high_color = hex_to_rgb(high_color)

    return plotly.colors.find_intermediate_color(
        lowcolor=low_color,
        highcolor=high_color,
        intermed=((intermed - low_cutoff) / (high_cutoff - low_cutoff)),
        colortype="rgb",
    )




cols = get_color("Viridis", np.linspace(0, 1, 5))
map_cols = np.linspace(0, 1, 5)
colorscale=[]
for i,val in enumerate(map_cols):
    colorscale.append([val, cols[i]])
    



# Simplify Plotting Schemes:
col1 =  px.colors.qualitative.Plotly[0]
col2 =  px.colors.qualitative.Plotly[1]
col3 =  px.colors.qualitative.Plotly[2]
col4 =  px.colors.qualitative.Plotly[3]
col5 =  px.colors.qualitative.Plotly[4]
col6 =  px.colors.qualitative.Plotly[5]


In [ ]:
def legend_as_annotation(fig, den_model_string, color_it, x_annot, y_annot):
    fig.add_annotation(
            x=x_annot,
            y=y_annot,
            xref="x domain",
            yref="y domain",
            showarrow=False,
            text=den_model_string,
            font=dict(
                size=16,
                color="#ffffff"
                ),
            align="center",
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor=color_it,
            opacity=0.9
            )

    return(fig)

# Plot Density

In [ ]:
def Plot_Densitycomparison(fig, obj_m1, plot_num):

    if plot_num == 0:
        col = col1
        x_annot = x_annot_val
        y_annot1 = 1
        y_annot2 = .9
        m_size = 4.5
    elif plot_num == 1:
        x_annot = x_annot_val
        y_annot1 = .8
        y_annot2 = .7
        col = col2
        m_size = 4
    elif plot_num == 2:
        x_annot = x_annot_val
        y_annot1 = .5
        y_annot2 = .5
        col = col3
        m_size = 3.5
    elif plot_num == 3:
        x_annot = x_annot_val
        y_annot1 = .2
        y_annot2 = .3
        col = col4
        m_size = 3
    elif plot_num == 4:
        x_annot = x_annot_val
        y_annot1 =  0 
        y_annot2 = .1
        col = col5
        m_size = 3
    elif plot_num == 5:
        x_annot = x_annot_val
        y_annot1 =  0 
        y_annot2 = .1
        col = col6
        m_size = 3



    model_m1 = obj_m1.__dict__['global_params']['den_model']
    for ii,arc in enumerate(obj_m1.__dict__['global_params']['arc_input'][:]):
        
        
        #### INDEX THE DENSITY DF correctly
        vals  = np.arange(obj_m1.__dict__['Density'][arc].index[0],obj_m1.__dict__['Density'][arc].index[-1]+1)
        df = obj_m1.__dict__['Density'][arc].set_index('Date',drop=False ) 
        df['i_vals'] = vals
        index_date = df.loc[df.index.max()]['i_vals'].min()

        
        str_run_param = 'run_parameters'+ arc
        final_iter = obj_m1.__dict__[str_run_param]['str_iteration']
        i_arc = ii+1
        
        print('----',model_m1,'----')
        print('     mean:    ',np.mean(obj_m1.Density[arc]['rho (kg/m**3)']),'----')
        print('     variance:',np.std(obj_m1.Density[arc]['rho (kg/m**3)']),'----')
        print()
        fig.add_trace(go.Scattergl(  x=obj_m1.Density[arc]['Date'][:index_date][:],
                                     y=obj_m1.Density[arc]['rho (kg/m**3)'][:index_date][:],
                                     name= model_m1+arc,
                                     mode='markers',
                                     opacity=1,
                                     marker=dict(
                                        color=col, 
                                        size=m_size,
                                                ),
                                     showlegend=False,
                                  ),
                                      secondary_y=False,
                                       row=1, col=1,
                                  )

#         (mean,rms,rms_about_zero) = STATS_residuals(data_resids['T'], 'in-track')

#         fig = add_stats_annotation(fig, model_m1+'<br>Mean='+ str(np.round(mean,4))+'<br>RMS='+ str(np.round(rms_about_zero,4)), col , x_annot, y_annot1)


    fig.update_yaxes( title="rho (kg/m**3)", type='log', exponentformat= 'power',row=1, col=1)
    fig.update_xaxes( title="Date", row=1, col=1)

    return(fig)



def orb_avg(den_df, arc):
    
    
    #### Find the index for the correct date
    vals  = np.arange(den_df[arc].index[0],den_df[arc].index[-1]+1)
    df = den_df[arc].set_index('Date',drop=False ) 
    df['i_vals'] = vals
    index_date = df.loc[df.index.max()]['i_vals'].min()
    
#     print('index_date', index_date)
    lat = np.asarray(den_df[arc]['Lat'][:index_date])
    time_pd = pd.to_datetime(den_df[arc]['Date'][:index_date])
    i = np.nonzero( lat[1:]*lat[0:-1]  <  np.logical_and(0 , lat[1:] > lat[0:-1] )  )
    i = i[0]

    d_avg = np.zeros(np.size(i))
    height_avg = np.zeros(np.size(i))
    
#     print('time_pd',time_pd)

    time_avg = []
    d_avg_rolling = []
    
    roll_avg_count = 0
    for j in range(np.size(i)-1):
        d_avg[j]      = np.mean(den_df[arc]['rho (kg/m**3)'  ][i[j] : i[j+1]-1  ]  )
        height_avg[j] = np.mean(den_df[arc]['Height (meters)'][i[j] : i[j+1]-1  ]  )
#         mean_time      = np.mean(time_pd[   i[j] : i[j+1]-1  ])
        t1 = pd.to_datetime(time_pd[ i[j]    ])
        t2 = pd.to_datetime(time_pd[ i[j+1]-1])
        datemiddle = pd.Timestamp(t1) + (pd.Timestamp(t2) - pd.Timestamp(t1)) / 2

        time_avg.append(datemiddle)

        if roll_avg_count ==1:
            d_avg_rolling.append(np.mean([ d_avg[j],  d_avg[j-1]]))
            roll_avg_count =0
            
        roll_avg_count+=1 
    d_avg_rolling.append(np.mean([ d_avg[j],  d_avg[j-1]]))
        
    return(time_avg, d_avg, d_avg_rolling )
    

def plot_density_orbit_avg(fig, obj_m1, plot_num ):
    
    model_m1 = obj_m1.__dict__['global_params']['den_model']
#     str_run_param = 'run_parameters'+ arc
    if plot_num == 0:
        col = col1
        x_annot = 1.09
        y_annot1 = 1
        y_annot2 = .9
        m_size = 3.5
    elif plot_num == 1:
        x_annot = 1.09
        y_annot1 = .8
        y_annot2 = .7
        col = col2
        m_size = 3.5
    elif plot_num == 2:
        x_annot = 1.09
        y_annot1 = .5
        y_annot2 = .5
        col = col3
        m_size = 3.5
    elif plot_num == 3:
        x_annot = 1.09
        y_annot1 = .2
        y_annot2 = .3
        col = col4
        m_size = 3.5
    elif plot_num == 4:
        x_annot = 1.09
        y_annot1 =  0 
        y_annot2 = .1
        col = col5
        m_size = 3.5
    elif plot_num == 5:
        x_annot = 1.09
        y_annot1 =  0 
        y_annot2 = .1
        col = col6
        m_size = 3.5


#     (f107a,
#     f107d,
#     date,
#     ap) = get_geomag_and_solarflux_data(obj_m1)


    for ii,arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):
        
        vals  = np.arange(obj_m1.__dict__['Density'][arc].index[0],obj_m1.__dict__['Density'][arc].index[-1]+1)
        df = obj_m1.__dict__['Density'][arc].set_index('Date',drop=False ) 
        df['i_vals'] = vals
        index_date = df.loc[df.index.max()]['i_vals'].min()

        
        time_avg,d_avg, d_avg_rolling = orb_avg(obj_m1.Density, arc)
        
        
        fig.add_trace(go.Scattergl(x=time_avg,
                                 y=d_avg_rolling,
#                                  y=d_avg,
#                                 name= ' Arc ' +str(i_arc) ,
                                 mode='markers',
                                 marker=dict(
                                 color=col,
                                 size=7,),
                                 showlegend=False,
                                   ),
                                   row=1, col=1,
                                   )
        

        fig.update_yaxes(type="log", exponentformat= 'power',row=1, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_yaxes(title_text="kg/m^3", row=1, col=1)
#     fig.update_yaxes(title_text="nT", row=2, col=1)
#     fig.update_yaxes(title_text="sfu", row=3, col=1)
    fig.update_layout(legend= {'itemsizing': 'constant'})
#     fig.update_layout(
#         font=dict(          size=18, ),
#         autosize=False,
#         width=900,
#         height=1000,)
    
    return(fig)



In [8]:
%load_ext autoreload
%autoreload 2


fig = make_subplots(
    rows=1, cols=1,
    subplot_titles=(['Sampled Orbit Densities']),
    vertical_spacing = 0.15)


# fig = Plot_Densitycomparison(fig, Obj_Geodyn['msis2'] ,2)
# fig = Plot_Densitycomparison(fig, Obj_Geodyn['dtm87'] ,3)
# fig = Plot_Densitycomparison(fig, Obj_Geodyn['jaachia71'] , 4)
# fig = Plot_Densitycomparison(fig, Obj_Geodyn['tiegcm_oc'] ,5)
fig = Plot_Densitycomparison(fig, Obj_Geodyn['hasdm_oc'] ,1)

fig.show(config=config)


# fig.write_image(plots_dir+ 'Density_along_orbit.jpg', format='jpg')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


NameError: name 'make_subplots' is not defined

In [ ]:
%load_ext autoreload
%autoreload 2


fig = make_subplots(
    rows=1, cols=1,
    subplot_titles=(['Orbit Averaged Densities']),
    vertical_spacing = 0.15)


fig = plot_density_orbit_avg(fig, Obj_Geodyn['msis2'] ,2)
fig = plot_density_orbit_avg(fig, Obj_Geodyn['dtm87'] ,3)
fig = plot_density_orbit_avg(fig, Obj_Geodyn['jaachia71'] , 4)
fig = plot_density_orbit_avg(fig, Obj_Geodyn['tiegcm_oc'] ,5)
fig = plot_density_orbit_avg(fig, Obj_Geodyn['hasdm_oc'] ,1)

fig.show(config=config)


# fig.write_image(plots_dir+ 'Density_along_orbit.jpg', format='jpg')


## Total RMS Timeseries

In [ ]:




def plot_TotalRMS_NTW(fig, obj_m1, plotname, plot_num):

    if plot_num == 0:
        col = col1
        x_annot = x_annot_val
        y_annot1 = 1
        y_annot2 = .9
        m_size = 3.5
    elif plot_num == 1:
        x_annot = x_annot_val
        y_annot1 = .8
        y_annot2 = .8
        col = col2
        m_size = 3.5
    elif plot_num == 2:
        x_annot = x_annot_val
        y_annot1 = .5
        y_annot2 = .7
        col = col3
        m_size = 3.5
    elif plot_num == 3:
        x_annot = x_annot_val
        y_annot1 = .2
        y_annot2 = .55
        col = col4
        m_size = 3.5
    elif plot_num == 4:
        x_annot = x_annot_val
        y_annot1 =  0 
        y_annot2 = .45
        col = col5
        m_size = 3.5
        
    elif plot_num == 5:
        x_annot = x_annot_val
        y_annot1 =  0 
        y_annot2 = .35
        col = col6
        m_size = 3.5
        
    import datetime
    doy_dates = []
    for i in obj_m1.__dict__['global_params']['arc_input']:
        doy_dates.append(datetime.datetime(int(i.split('.')[0]), 1, 1) + datetime.timedelta(int(i.split('.')[1]) - 1))
    

    text_ntw = []
    for itxt,txt in enumerate(obj_m1.__dict__['Statistics']['RMS_total_NTW']):
        text_ntw.append(str(np.round(obj_m1.__dict__['Statistics']['RMS_total_NTW'][itxt],3)))
    fig.add_trace(go.Scattergl(
                             x=doy_dates,
                             y=obj_m1.__dict__['Statistics']['RMS_total_NTW'],
                         name='NTW',
                         mode='markers+lines',
                         marker=dict(
                             color=col,
                             size=10,
                                    ),
                    text = text_ntw,
                    textposition = "top center",
                    textfont = {'color': [col]*(itxt+1)},
                         showlegend=False),
                         secondary_y=False,
                         row=1, col=1)
    
    fig = legend_as_annotation(fig, plotname, col, x_annot, y_annot2)
    fig.update_yaxes( title="Total RMS [meters]" ,type="linear" , exponentformat= 'power',row=1, col=1)
    fig.update_xaxes( title="Date"    ,row=1, col=1)


    return(fig)
    


In [ ]:
%load_ext autoreload
%autoreload 2

fig = make_subplots(
    rows=1, cols=1,
    subplot_titles=(['Total RMS in the NTW Coordinate System']),
    vertical_spacing = 0.1)



fig = plot_TotalRMS_NTW(fig,  Obj_Geodyn['msis2'] ,'msis2' , 2 )
fig = plot_TotalRMS_NTW(fig,  Obj_Geodyn['dtm87'] ,'dtm87' , 3 )
fig = plot_TotalRMS_NTW(fig,  Obj_Geodyn['jaachia71'] ,'jaachia71' , 4 )
fig = plot_TotalRMS_NTW(fig,  Obj_Geodyn['tiegcm_oc'] ,'tiegcm_oc' , 5 )
fig = plot_TotalRMS_NTW(fig,  Obj_Geodyn['hasdm_oc'] ,'hasdm_oc' , 1 )

fig.show(config=config)
# fig.write_image(plots_dir+ 'TotalRMS_NTW.jpg', format='jpg')


## Full NTW Residuals


In [ ]:
def plotBasicResids_ChooseCoords(fig, obj_m1, coords, plotname, plot_num):

    if plot_num == 0:
        col = col1
        x_annot = x_annot_val
#         y_annot1 = 1
        y_annot2 = .9
        m_size = 3.5
    elif plot_num == 1:
        x_annot = 1.09
#         y_annot1 = .8
        y_annot2 = .75
        col = col2
        m_size = 3.5
    elif plot_num == 2:
        x_annot = 1.09
#         y_annot1 = .5
        y_annot2 = .5
        col = col3
        m_size = 3.5
    elif plot_num == 3:
        x_annot = 1.09
#         y_annot1 = .2
        y_annot2 = .253
        col = col4
        m_size = 3.5
    elif plot_num == 4:
        x_annot = 1.09
#         y_annot1 =  0 
        y_annot2 = .1
        col = col5
        m_size = 3.5
    elif plot_num == 5:
        x_annot = 1.09
#         y_annot1 =  0 
        y_annot2 = -.0810
        col = col6
        m_size = 3.5


    model_m1 = obj_m1.__dict__['global_params']['den_model']
    for ii,arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):
        
        data_resids = obj_m1.__dict__['OrbitResids'][arc]['resids']


        fig.update_layout(title=''.join(coords)+' Orbit Residuals',
                        autosize=True,
                        font=dict(size=14),
                        legend= {'itemsizing': 'constant'})

        for i,val in enumerate(coords):
            fig.add_trace(go.Scattergl(
                                     x=data_resids['Date'][::50],
                                     y=data_resids[val][::50],
                                 name=val+' '+model_m1,
                                 mode='markers',
                                 marker=dict(
                                     color=col,
                                     size=m_size,
                                            ),
                                 showlegend=False),
                                 secondary_y=False,
                                 row=i+1, col=1)

#             (mean,rms,rms_about_zero) = STATS_residuals(data_resids[val], val+plotname)

#             fig = add_stats_annotation(fig, plotname+'<br>Mean='+ str(np.round(mean,4))+'<br>RMS='+ str(np.round(rms_about_zero,4)), col , x_annot, y_annot1)
    fig = legend_as_annotation(fig, plotname, col, x_annot, y_annot2)

        
    fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=1, col=1)
    fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=2, col=1)
    fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=3, col=1)
    fig.update_xaxes( title="Date",  row=3, col=1)

    return(fig)


In [ ]:
plotBasicResids_ChooseCoords
coords = ['N','T','W']

fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(['N (Normal) ','T (In-Track)','W (Cross-Track)']),
    vertical_spacing = 0.1)

# msis2','msis86','dtm87', 'jaachia71']
fig = plotBasicResids_ChooseCoords(fig, Obj_Geodyn['msis2']   , coords,  'msis2' , 2)
fig = plotBasicResids_ChooseCoords(fig, Obj_Geodyn['dtm87']   , coords,  'dtm87' , 3)
fig = plotBasicResids_ChooseCoords(fig, Obj_Geodyn['jaachia71']   , coords,  'jaachia71' , 4)
fig = plotBasicResids_ChooseCoords(fig, Obj_Geodyn['tiegcm_oc']   , coords,  'tiegcm_oc' , 5)
fig = plotBasicResids_ChooseCoords(fig, Obj_Geodyn['hasdm_oc']   , coords,  'hasdm_oc' , 1)



fig.update_layout(title=''.join(coords)+' Orbit Residuals',
                autosize=True,
                font=dict(size=14),
                legend= {'itemsizing': 'constant'})

fig.update_layout(
        autosize=False,
        width=800,
        height=900)
fig.show(config=config)


